### 3.1 Predict Messages

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)

# 가상의 대화 설정
messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Korean."
    ),
    AIMessage(content="안녕! 나는 챗봇이야."),
    HumanMessage(content="What is the distance between Korea and Japan? Also, what is your name?"),
]

chat.predict_messages(messages)

AIMessage(content='한국과 일본 사이의 거리는 약 900km입니다. 제 이름은 지리 전문가 챗봇이에요.')

### 3.2 Prompt Templates

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}?",
)

prompt = template.format(country_a="Korea", country_b="Vietnam")

chat.predict(prompt)

'The distance between Korea and Vietnam is approximately 2,300 kilometers (1,429 miles) when measured in a straight line.'

In [3]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕! 나는 {name}이야."),
        ("human", "What is the distance between {country_a} and {country_b}? also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language="Korean",
    name="chatbot",
    country_a="Korea",
    country_b="France",
)

chat.predict_messages(prompt)

AIMessage(content='한국과 프랑스 사이의 거리는 대략 9000km입니다. 제 이름은 Chatbot이에요.')

### 3.3 OutputParser and LCEL(LangChain Expression Language)

In [4]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    # must define parse function
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()
p.parse('hi,how,are,you')

['hi', 'how', 'are', 'you']

In [5]:
# without LCEL

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}."),
    ]
)

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [6]:
# with LCEL
# make a chain of chat model, OutputParser and template.

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lowercase. Do NOT reply with anything else."),
        ("human", "{question}."),
    ]
)

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle']

### 3.4 Chaining Chains

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

# streaming: LLM의 응답이 생성되는걸 볼 수 있게 해줌
# StreamingStdOutCallbackHandler: console에서 응답의 진행을 볼 수 있음
chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [8]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

In [9]:
final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and easy recipe for Chicken Tikka Masala.

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tsp ground cumin
- 2 tsp paprika
- 1 tsp ground turmeric
- 1 tsp garam masala
- 1 tsp ground coriander
- 1/2 tsp cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, combine yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Thread the marinated chicken piece

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe.\n   - Drain the tofu and press it to remove excess water. You can do this by wrapping the tofu block in a clean kitchen towel or paper towels and placing a heavy object on top for about 15-30 minutes.\n   - Cut the tofu into bite-sized cubes and proceed with marinating it as you would with the chicken. Tofu absorbs flavors well, so the marination process is crucial for a tasty dish.\n   - You can either bake the marinated tofu in the oven like you would with the chicken or pan-fry it until golden brown before adding it to the sauce.\n\n2. **Paneer**:\n   - Paneer is a fresh cheese commonly used in Indian cuisine.\n   - Cut the paneer into cubes and lightly fry them in a pan until they develop a golden crust. Thi